### Notebook to be used to perform focus sweep using X images, then processing them to find the best focus. In this case we're using the 0th order image but this can be modified.

In [1]:
import numpy as np
from lsst.ts import salobj
import asyncio
from astropy.io import fits

import warnings
#import matplotlib.pyplot as plt  # imported as py above
from astropy.modeling import models, fitting
from scipy.ndimage.filters import gaussian_filter
from matplotlib import pyplot as plt
#%matplotlib ipympl
plt.rcParams['figure.figsize'] = [7, 6]

import lsst.daf.persistence as dafPersist
import matplotlib.pyplot as plt
%matplotlib inline
import lsst.afw.display as afwDisplay
#afwDisplay.setDefaultBackend('firefly')
afwDisplay.setDefaultBackend('matplotlib')
import time
import lsst.afw.cameraGeom.utils as cameraGeomUtils
import lsst.geom

import os
import lsst.log
lsst.log.usePythonLogging()

import logging
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s')
logger = logging.getLogger('image_display_notebook')

In [2]:
# Setup the butler
accs_images = True
if accs_images:
    repo = os.path.join("/home/saluser/ingest/accs/")#, mapper={'calibRoot': "/home/saluser/ingest/dmcs/CALIB"})
    butler = dafPersist.Butler(repo)
else:
    repo = os.path.join("/home/saluser/ingest/dmcs/")#, mapper={'calibRoot': "/home/saluser/ingest/dmcs/CALIB"})
    butler = dafPersist.Butler(repo)
#raw = butler.get("raw", visit=2019053100020)
#image = raw.getImage().array

2019-11-13 01:13:42,242 CameraMapper INFO     Loading exposure registry from /home/saluser/ingest/accs/registry.sqlite3
2019-11-13 01:13:42,244 CameraMapper INFO     Loading calib registry from /home/saluser/ingest/accs/CALIB/calibRegistry.sqlite3


In [3]:
#Setup the remotes

d = salobj.Domain()
ATArchiver = salobj.Remote(d, 'ATArchiver')
#await ATArchiver.start_task
ATCamera = salobj.Remote(d, 'ATCamera')
#await ATCamera.start_task
LinearStage1 = salobj.Remote(d, 'LinearStage',1)
#await LinearStage1.start_task
ATSpectrograph = salobj.Remote(d, 'ATSpectrograph')
await asyncio.gather(ATArchiver.start_task, ATCamera.start_task, 
                     LinearStage1.start_task, ATSpectrograph.start_task)

2019-11-13 01:13:57,797 ATArchiver   INFO     Read historical data in 7.59 sec
2019-11-13 01:13:57,935 LinearStage  INFO     Read historical data in 7.72 sec
2019-11-13 01:13:57,936 ATSpectrograph INFO     Read historical data in 7.72 sec
2019-11-13 01:13:58,096 ATCamera     INFO     Read historical data in 7.89 sec


[None, None, None, None]

In [5]:
from lsst.ts.standardscripts.auxtel.latiss import LATISS

latiss = LATISS(salobj.Remote(domain=d, name="ATCamera"), 
                salobj.Remote(domain=d, name="ATSpectrograph"))

await asyncio.gather(latiss.atcam.start_task, latiss.atspec.start_task)

2019-11-12 23:09:58,011 ATSpectrograph WARNING  RemoteEvent(ATSpectrograph, 0, heartbeat) falling behind; read 100 messages
2019-11-12 23:09:58,081 ATSpectrograph WARNING  RemoteEvent(ATSpectrograph, 0, heartbeat) falling behind; read 100 messages
2019-11-12 23:09:58,149 ATArchiver   WARNING  RemoteEvent(ATArchiver, 0, heartbeat) falling behind; read 100 messages
2019-11-12 23:09:58,216 ATArchiver   WARNING  RemoteEvent(ATArchiver, 0, heartbeat) falling behind; read 100 messages
2019-11-12 23:09:58,284 ATCamera     WARNING  RemoteTelemetry(ATCamera, 0, wrebPower) falling behind; read 15 messages
2019-11-12 23:09:58,294 ATCamera     WARNING  RemoteTelemetry(ATCamera, 0, wrebPower) falling behind; read 15 messages
2019-11-12 23:09:58,313 ATCamera     WARNING  RemoteTelemetry(ATCamera, 0, wreb) falling behind; read 14 messages
2019-11-12 23:09:58,324 ATCamera     WARNING  RemoteTelemetry(ATCamera, 0, wreb) falling behind; read 14 messages
2019-11-12 23:09:58,353 ATCamera     WARNING  Remo

[None, None]

### Leave this cell for state transitions 

In [4]:
await salobj.set_summary_state(ATSpectrograph, salobj.State.STANDBY, settingsToApply='default')

[<State.FAULT: 3>, <State.STANDBY: 5>]

## Setup ATSpectrograph

In [18]:
tmp = await ATSpectrograph.cmd_homeLinearStage.start()
print(tmp)

2019-11-13 00:03:26,884 ATCamera     WARNING  RemoteTelemetry(ATCamera, 0, vacuum) falling behind; read 18 messages
2019-11-13 00:03:26,898 ATCamera     WARNING  RemoteTelemetry(ATCamera, 0, vacuum) falling behind; read 18 messages
2019-11-13 00:03:26,913 ATCamera     WARNING  RemoteTelemetry(ATCamera, 0, vacuum) falling behind; read 18 messages


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1127146029, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result="Failed: 'CSC' object has no attribute 'home_gs'")

In [25]:
await latiss.setup_atspec(grating=2, filter=2, linear_stage=70.0)

2019-11-13 00:06:36,339 ATArchiver   WARNING  RemoteEvent(ATArchiver, 0, heartbeat) falling behind; read 17 messages
2019-11-13 00:06:36,375 ATArchiver   WARNING  RemoteEvent(ATArchiver, 0, heartbeat) falling behind; read 17 messages
2019-11-13 00:06:36,401 ATSpectrograph WARNING  RemoteEvent(ATSpectrograph, 0, heartbeat) falling behind; read 17 messages
2019-11-13 00:06:36,425 ATSpectrograph WARNING  RemoteEvent(ATSpectrograph, 0, heartbeat) falling behind; read 17 messages
2019-11-13 00:06:36,445 ATSpectrograph WARNING  RemoteEvent(ATSpectrograph, 0, heartbeat) falling behind; read 17 messages


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=245052368, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result="Failed: Controller not ready: Received 'b'I''...")

In [15]:
tmp = await ATSpectrograph.evt_reportedLinearStagePosition.next(flush=False,timeout=5)
print(tmp)

2019-11-12 23:59:12,057 ATCamera     WARNING  RemoteTelemetry(ATCamera, 0, wrebPower) falling behind; read 29 messages
2019-11-12 23:59:12,100 ATCamera     WARNING  RemoteTelemetry(ATCamera, 0, wrebPower) falling behind; read 29 messages
2019-11-12 23:59:12,178 ATCamera     WARNING  RemoteTelemetry(ATCamera, 0, wrebPower) falling behind; read 29 messages


TimeoutError: 

## Setup Monochromator

In [ ]:
wavelength = 455

In [ ]:
# ATMonochromator.cmd_changeWavelength.set(wavelength=wavelength)
# try:
#     asyncio.get_event_loop().run_until_complete(ATMonochromator.cmd_changeWavelength.start())
# except AckError as ack_err:
#     print(f"Failed with ack.result={ack_err.ack.result}")

In [ ]:
#ATMonochromator.cmd_selectGrating.set(gratingType=0)
#asyncio.get_event_loop().run_until_complete(ATMonochromator.cmd_selectGrating.start())

In [ ]:
#ATMonochromator.cmd_changeSlitWidth.set(slitWidth=0.2,
#                          slit=SALPY_ATMonochromator.ATMonochromator_shared_Slit_FrontEntrance)
#asyncio.get_event_loop().run_until_complete(ATMonochromator.cmd_changeSlitWidth.start())

In [ ]:
#ATMonochromator.cmd_changeSlitWidth.set(slitWidth=0.2,
#                          slit=SALPY_ATMonochromator.ATMonochromator_shared_Slit_FrontExit)
#asyncio.get_event_loop().run_until_complete(ATMonochromator.cmd_changeSlitWidth.start())

## Setup the Linear Stage (focuser)

In [ ]:
pos=None
pos=await LinearStage1.tel_position.next(flush=True, timeout=5)
print('Current stage position is {:0.4f} [mm]'.format(pos.position))

In [ ]:
# Home the stage (only necessary if power was off)
if True:
    await LinearStage1.cmd_getHome.start()
else:
    print('Ignoring')

In [ ]:
# Move the stage
lin_stage_pos=27.5
LinearStage1.cmd_moveAbsolute.set(distance=lin_stage_pos)
try:
    asyncio.get_event_loop().run_until_complete(LinearStage1.cmd_moveAbsolute.start())
except AckError as ack_err:
    print(f"Failed with ack.result={ack_err.ack.result}")

In [ ]:
wavelength=455
# 33 is best focus for 0th order for grating and no filter (632nm)
# 33.85 best forcus for 0th order for no grating and no filter (632nm)
focus_center=27.5
dx=5.
step=1.5
focus_demands = np.arange(focus_center-dx, focus_center+dx, step)
#focus_vals = np.arange(68.4, 71.3, 0.3) - coarse
print(focus_demands)

In [ ]:
image_list=[]
focus_vals=np.empty(len(focus_demands))
for i,lin_stage_pos in enumerate(focus_demands):
    print('Setting focus of {} [mm]'.format(lin_stage_pos))
    
    # Move focus stage
    LinearStage1.cmd_moveAbsolute.set(distance=lin_stage_pos)
    try:
        await LinearStage1.cmd_moveAbsolute.start()
    except salobj.AckError as ack_err:
        print(f"Failed with ack.result={ack_err.ack.result}")
    
    focus_vals[i]=(await LinearStage1.tel_position.next(flush=True, timeout=5)).position
    print('Linear stage demand {}, actual {}:'.format(lin_stage_pos, focus_vals[i]))
    
    # Take image
    group_id='Focus'+str(wavelength)+str(lin_stage_pos)
    ATCamera.cmd_takeImages.set(expTime=0.2, shutter=1, numImages=1, imageSequenceName=group_id)
    ATCamera.evt_endReadout.flush()
    try:
        await ATCamera.cmd_takeImages.start()
    except salobj.AckError as ack_err:
        print(f"Failed with ack.result={ack_err.ack.result}")

    endReadout = await ATCamera.evt_endReadout.next(flush=False, timeout=30)
    print('Wrote file {}'.format(endReadout.imageName) )
    
    image_list.append(endReadout.imageName)

In [ ]:
for index, img in enumerate(image_list):
    print('{}, {:0.2f}'.format(img, focus_vals[index]))

In [ ]:
#tmp = asyncio.get_event_loop().run_until_complete(ATArchiver.evt_processingStatus.next(flush=False, timeout=30))
#tmp.description

In [ ]:
import importlib
import utils.processExposure 
importlib.reload(utils.processExposure)
from utils.processExposure import processExposure


import utils.grabATImage 
importlib.reload(utils.grabATImage)
from utils.grabATImage import grabATImage

In [ ]:
# parse out visitID from filename - this is highly annoying
tmp=endReadout.imageName.split('_')
prefix=tmp[2] # dayobs without the dashes

# Don't remember why I used int here... whitespace? 
# surely fixable but bigger fish.
suffix='{:05d}'.format(int(tmp[3].split('-')[0])) # SEQNUM, but need to trim extra 0 in obsid
visitID = int((prefix+suffix))
dataId1 = {'visit': visitID}
print(visitID)

# Grab image from butler, but need to wait to ingestion so use this polling function
exposure = await grabATImage(visitID, repo, timeout = 40, poll_freq_hz=2)

# do ISR correction
isr_corr_exposure = processExposure(exposure, repo=repo)

In [ ]:
# Only here for me to debug, should be up top with other declarations
import importlib
import utils.findNarrowbandRonchiPeaks
importlib.reload(utils.findNarrowbandRonchiPeaks)
from utils.findNarrowbandRonchiPeaks import findNarrowbandRonchiPeaks

import utils.fitExposure
importlib.reload(utils.fitExposure)
from utils.fitExposure import fit2DGaussian

import utils.calc_CofM
importlib.reload(utils.calc_CofM)
from utils.calc_CofM import calc_CofM

import utils.calc_encircled_energy
importlib.reload(utils.calc_encircled_energy)
from utils.calc_encircled_energy import calc_encircled_energy

In [ ]:
# Source detection libraries
from lsst.meas.algorithms.detection import SourceDetectionTask
import lsst.afw.table as afwTable

# create the output table for source detection
schema = afwTable.SourceTable.makeMinimalSchema()
config = SourceDetectionTask.ConfigClass()
config.thresholdValue = 10  # detection threshold after smoothing
sourceDetectionTask = SourceDetectionTask(schema=schema, config=config)

In [ ]:
# I don't like looping but I don't know how to handle multiple files yet!
# Declare approximation of where the zero-order star is
zeroth_order_estimate = lsst.geom.Point2D(1700,1960)
zeroth_order_estimate = lsst.geom.Point2D(2075,2025) # low res grating
zeroth_order_estimate = lsst.geom.Point2D(2260,2050) #empty (632)
zeroth_order_estimate = lsst.geom.Point2D(2010,2175) #empty (455)
zeroth_order_star_BBox= lsst.geom.Box2I.makeCenteredBox(zeroth_order_estimate, lsst.geom.Extent2I(200,200)) 

fit_data=[]

for index, img_name in enumerate(image_list):
    # parse out visitID from filename - this is highly annoying
    print('Processing file {} of {}, filename={}, focus={}'.format(index,len(image_list), img_name, focus_vals[index]))
    tmp=img_name.split('_')
    prefix=tmp[2] # dayobs without the dashes
    # Don't remember why I used int here... whitespace? 
    # surely fixable but bigger fish.
    suffix='{:05d}'.format(int(tmp[3].split('-')[0])) # SEQNUM, but need to trim extra 0 in obsid
    visitID = int((prefix+suffix))
    dataId1 = {'visit': visitID}
    #multi_file_dataset[i]['visitID']=visitID
    
    #exposure = butler.get('raw', **dataId1)
    exposure = await grabATImage(visitID, repo, timeout = 40, poll_freq_hz=2)
    # do ISR correction
    isr_corr_exposure = processExposure(exposure, repo=repo, defects=None) #bias=None
    
    # Find all sources in the image
    tab = afwTable.SourceTable.make(schema)
    # Find the correct sources using just a strip
    search_center_pt = lsst.geom.Point2D(zeroth_order_estimate.getX(),2000)
    search_strip = lsst.geom.Box2I.makeCenteredBox(search_center_pt, lsst.geom.Extent2I(500,3999)) 
    result = sourceDetectionTask.run(tab, isr_corr_exposure[search_strip], sigma=2.0)
    
    # wavelength solution is bad for the fiberSpectrograph, but close enough for this to work
    # can use the monochromator wavelength which is better, but that's not the correct way in the long run

    if False: # display the image in firefly
        plt.close('all')
        disp = afwDisplay.Display(2, reopenPlot=True)
        disp.setMaskPlaneColor('SAT', afwDisplay.IGNORE)
        disp.setImageColormap('gray')
        disp.scale('linear', 'minmax')
        peak0_subim = isr_corr_exposure.subset(zeroth_order_star_BBox)
        disp.mtv(peak0_subim, title='visit = {}'.format(visitID))        
        #disp.scale('linear', 'zscale')
        #disp.mtv(isr_corr_exposure, title='visit = {}'.format(visitID))
        #cgUtils.overlayCcdBoxes(isr_corr_exposure.getDetector(), isTrimmed=True, display=disp)
        break
        
    
    dispersion = (1/0.6358) # pixels/nm
    spectral_position_angle=0.0107 # radians clockwise from top

    # Find 0th and +/- 1 order peaks 
    sources = result.sources
    
    #center_source, peak1, peak2 = findNarrowbandRonchiPeaks(sources, zeroth_order_star_BBox, wavelength, dispersion, spectral_position_angle)
    
    dispersion = None
    center_source = findNarrowbandRonchiPeaks(sources, zeroth_order_star_BBox, wavelength, dispersion, spectral_position_angle)
    # Fit peaks
    # zeroth order
    # variables names are weird here because I can't think of a clever way to have -1 and +1 as variable names
    
    bbox0 = lsst.geom.Box2I.makeCenteredBox(center_source.getFootprint().getCentroid(), lsst.geom.Extent2I(50,50)) 
    peak0_subim = isr_corr_exposure.subset(bbox0)
    p0, x0 , y0 = fit2DGaussian(peak0_subim, plot=True)
    p0_x_CofM, p0_y_CofM = calc_CofM(peak0_subim) # 2167,3372

# Calculate EE and CofM
    p0_EE_rad50_pix, p0_EE_rad67_pix, p0_EE_rad80_pix = calc_encircled_energy(peak0_subim, plot=False) 
    fit_data.append(p0_EE_rad80_pix)

#     # offset to test
#     bbox1 = lsst.geom.Box2I.makeCenteredBox(peak1.getFootprint().getCentroid(), lsst.geom.Extent2I(50,50)) 
#     peak1_subim = isr_corr_exposure.subset(bbox1)
#     p1, x1 , y1 = fit2DGaussian(peak1_subim, plot=True)
#     p1_x_CofM, p1_y_CofM = calc_CofM(peak1_subim) # 2167,3372
#     fit_data.append(p1_EE_rad67_pix)

#     # Calculate EE and CofM
#     p1_EE_rad50_pix, p1_EE_rad67_pix, p1_EE_rad80_pix = calc_encircled_energy(peak1_subim, plot=False) 
    
#     fit_data.append(p1_EE_rad80_pix)
#     bbox2 = lsst.geom.Box2I.makeCenteredBox(peak2.getFootprint().getCentroid(), lsst.geom.Extent2I(50,50)) 
#     peak2_subim = isr_corr_exposure.subset(bbox2)
#     p2, x2 , y2 = fit2DGaussian(peak2_subim, plot=False)
#     p2_x_CofM, p2_y_CofM = calc_CofM(peak2_subim) # 2167,3372

#     # Calculate EE and CofM
#     p0_EE_rad50_pix, p0_EE_rad67_pix, p0_EE_rad80_pix = calc_encircled_energy(peak0_subim, plot=False) 
#     p1_EE_rad50_pix, p1_EE_rad67_pix, p1_EE_rad80_pix = calc_encircled_energy(peak1_subim, plot=False) 
#     p2_EE_rad50_pix, p2_EE_rad67_pix, p2_EE_rad80_pix = calc_encircled_energy(peak2_subim, plot=False) 

#     #  Now use multi_file_dataset
#     multi_file_dataset[i]['Gauss_x_peak'] = (p0.x_mean.value, p1.x_mean.value, p2.x_mean.value)
#     multi_file_dataset[i]['Gauss_y_peak'] = (p0.y_mean.value, p1.y_mean.value, p2.y_mean.value)
#     multi_file_dataset[i]['Gauss_xsigma_pix'] = (p0.x_stddev.value, p1.x_stddev.value, p2.x_stddev.value) 
#     multi_file_dataset[i]['Gauss_ysigma_pix'] = (p0.y_stddev.value, p1.y_stddev.value, p2.y_stddev.value)
#     multi_file_dataset[i]['x_CofM'] = (p0_x_CofM, p1_x_CofM, p2_x_CofM)
#     multi_file_dataset[i]['y_CofM'] = (p0_y_CofM, p1_y_CofM, p2_y_CofM)
#     multi_file_dataset[i]['EE50_pix'] = (p0_EE_rad50_pix, p1_EE_rad50_pix, p2_EE_rad50_pix)
#     multi_file_dataset[i]['EE67_pix'] = (p0_EE_rad67_pix, p1_EE_rad67_pix, p2_EE_rad67_pix)
#     multi_file_dataset[i]['EE80_pix'] = (p0_EE_rad80_pix, p1_EE_rad80_pix, p2_EE_rad80_pix)

In [ ]:
tmp=None
if tmp != None:
    print('Here')

In [ ]:
fit_data

In [ ]:
# Fit a line to the profile
from scipy.optimize import curve_fit


xdata=focus_vals#[0:4]
ydata = fit_data #[3:9]

def parabola(x,b, x0, a):
    return b + a*(x-x0)**2 

popt,pcov = curve_fit(parabola, xdata, ydata)#, p0=[3.0, 75.5 , 1])

plt.close('all')
plt.figure(figsize=(13, 5))
plt.ylabel('Encircled Energy [pix]')
plt.plot(xdata, ydata, '.')
x=np.arange(np.min(xdata), np.max(xdata), np.abs(np.max(xdata) - np.min(xdata))/100 )
plt.plot(x, parabola(x, *popt))
plt.title('Encircled Energy [pix]')
plt.xlabel('Focus position [mm]')
plt.show()
plt.close()

print('Best focus occurs at {} [mm]'.format(popt[1]))


In [ ]:
26*25.4

In [ ]:
pup_diam=25e-3; wave=632e-9
f_coll=113e-3; 

f_tel=550e-3 # 400mm nominal - measured around 25-26 inches
pinhole_diam=2e-6
pix_scl=10 # [um/pix]

diff_limit=1e6*wave/pup_diam*f_coll
print('Diffraction Limit in object space (f=125mm, D=25mm, wave=632nm) is: {0:3f} [um]'.format(diff_limit))
mag=f_tel/f_coll
print('Image diameter of Diffraction Limit is: {0:3f} [um]'.format(mag*diff_limit/pix_scl))
print('Magnification is f_tel/f_col {}:'.format(mag))
im_size=1e6*mag*pinhole_diam
print('Expected image diameter for {0:3f} diameter pinhole: {1:3f} [um]'.format(pinhole_diam*1e6, im_size))
print('80% EE radius in pixels ~{}'.format(im_size/2/pix_scl))

In [ ]:
27*12/550.*2 / 10e-3

In [ ]:
27*4/550.*2 / 10e-3